# import + functions

import os

In [ ]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt  
import json

# main

In [11]:
# pull in the data and calculate deltas
data_path = r"Z:\projects\sequence_squad\revision_data\emmett_revisions\sleep_wake_link_data\replay_to_behaviour\\"

replay_type = 'sequential_NREM_and_REM_sleep'
## 




In [37]:
for mir in os.listdir(data_path)[3::]:
    print(f"Processing {mir}")
    current_path = os.path.join(data_path, mir)
    # only bother loading any of the data if there is replay data saved out. 
    if 'replay' in os.listdir(current_path):
        replay_path = os.path.join(current_path, 'replay',replay_type)
        # load replay json 
        json_path = replay_path + r'\replay_data_variables.json'
        with open(json_path, 'r') as file:
            replay_data = json.load(file)
            print(f"Loaded replay data for {mir} from {json_path}")
            
        # load in beahvioural data
        for file in os.listdir(current_path):
            if 'day2' in file:
                day2_path = os.path.join(current_path, file)
                if 'processed' in os.listdir(day2_path):
                    processed_poke_data_2 = pd.read_csv(os.path.join(day2_path, 'processed', 'processed_poke_data_2.csv'))
                    print(f"Loaded poke data 2 for {day2_path}")
                    try:
                        processed_tracking_data_2 = pd.read_csv(os.path.join(day2_path, 'processed', 'processed_tracking_data.csv'))
                        Tracking = True
                        print(f"Tracking data 2 found for {day2_path}")
                    except:
                        print(f"Tracking data not found for {day2_path}")
                        Tracking = False
        for file in os.listdir(current_path):
            if 'day1' in file:
                day1_path = os.path.join(current_path, file)
                if 'processed' in os.listdir(day1_path):
                    processed_poke_data_1 = pd.read_csv(os.path.join(day1_path, 'processed', 'processed_poke_data_1.csv'))
                    print(f"Loaded poke data 1 for {day1_path}")
                    try:
                        processed_tracking_data_1 = pd.read_csv(os.path.join(day1_path, 'processed', 'processed_tracking_data.csv'))
                        Tracking = True
                        print(f"Tracking data 1 found for {day1_path}")
                    except:
                        print(f"Tracking data not found for {day1_path}")
                        Tracking = False
                        
    break
        

Processing EJT136_1_3
Loaded replay data for EJT136_1_3 from Z:\projects\sequence_squad\revision_data\emmett_revisions\sleep_wake_link_data\replay_to_behaviour\\EJT136_1_3\replay\sequential_NREM_and_REM_sleep\replay_data_variables.json
Loaded poke data 2 for Z:\projects\sequence_squad\revision_data\emmett_revisions\sleep_wake_link_data\replay_to_behaviour\\EJT136_1_3\day2_12-11-2021
Tracking data 2 found for Z:\projects\sequence_squad\revision_data\emmett_revisions\sleep_wake_link_data\replay_to_behaviour\\EJT136_1_3\day2_12-11-2021
Loaded poke data 1 for Z:\projects\sequence_squad\revision_data\emmett_revisions\sleep_wake_link_data\replay_to_behaviour\\EJT136_1_3\day1_11-11-2021
Tracking data 1 found for Z:\projects\sequence_squad\revision_data\emmett_revisions\sleep_wake_link_data\replay_to_behaviour\\EJT136_1_3\day1_11-11-2021


In [51]:
## calculate behvaioural deltas
delta_behavioural_data = processed_poke_data_2 - processed_poke_data_1
if Tracking:
    delta_tracking_data = processed_tracking_data_2 - processed_tracking_data_1

In [54]:
delta_behavioural_data

,num_trials,correct_transit_rate,error_transit_rate,neutral_transit_rate,perf_score,total_transitions,transits_per_reward,seconds_per_reward
0,92,-0.080249,0.075162,0.005087,-0.286907,1752,2.116936,-6.390712


In [53]:
delta_tracking_data

,sequence_motif,mean_h_distance_from_av_mm,std_h_distance_from_av_mm,mean_dtw_distance_from_av_mm,std_dtw_distance_from_av_mm,mean_movement_speed_mm_s,std_movement_speed_mm_s
0,0,0.702269,0.584591,0.084063,-0.042136,13.671231,3.446381
1,0,-0.231616,-0.070824,-0.239316,-0.108740,5.740315,1.322276
2,0,-0.193072,0.038966,-1.024002,-0.483729,14.623149,-2.853868
3,0,0.102085,0.424722,-0.564206,-0.209347,15.097801,-1.185813


In [ ]:
# work out what the next steps are...I need to talk to chat gpt
# essentially I want to know if replay features can predict the behavioural deltas? 
# how do I do a regression with muliple predictors and mulitple outcomes? 
# maybe I cant? Maybe the predictor (replay) variables need to be combined into a single variable? or I need to do a GLM for each? 

In [ ]:
# looks like I want to use a multivariate regression 
# I will use the R squared as a meausre of predictive power of the model, and compare this to shuffled replay features. 